# Week 1: Prompting

---

In this tutorial, we will:  
1. Demonstrate the limitations of LLMs with examples.
2. Build a RAG using LangChain and LangGraph.
3. Enable the LLM's web search functionality.  




## Table of Contents

**It is recommended to use the TOC in the sidebar in Colab.**

- [Install dependencies](#install-dependencies)
- [Setup Google Gemini API Key](#setup-google-gemini-api-key)
  - [Register](#register)
  - [Enter your API Key for this Colab](#enter-your-api-key-for-this-colab)
  - [The LLM Model we're going to use](#the-llm-model-were-going-to-use)


# 1.Install dependencies

In [2]:
%%capture
%%shell
pip install google-genai
pip install langchain langchain-community langchain-google-genai

# 2.Setup Google Gemini API Key

## 2.1 Register

1. Visit the website: [https://aistudio.google.com/](https://aistudio.google.com/) and log in with your Google account.

2. Click on "Get API key."

<img src="https://lh3.google.com/u/0/d/16x6gM2WAvmbOkayzKGtfnavNPFp3Pgz4">

3. Agree to the terms of use by selecting only the first checkbox.

<img src="https://lh3.google.com/u/0/d/1bN1iR64XS-ibE-L47Dy_nQUu-idh24eS">

4. Generate API Keys  

  Click the "Create API key" button. You will see two options:  
- The first option, **"Create API key in new project"**, will create a new GCP (Google Cloud Platform) project and generate a new key.  
- The second option allows you to select an existing GCP project where the key will be created, if you've used GCP before.  

If this is your first time using GCP, select the first option.

<img src="https://lh3.google.com/u/0/d/1yhNB5BT6Wtobxjhlb9cAJ4FGUv75CG3p">

5. Copy the Google Gemini API key.

<img src="https://lh3.google.com/u/0/d/1J9sO5UMCz_ylNO27ku8vJr9_rKp63k5T">

## 2.2 Enter your API Key for this Colab

In [3]:
import os
import getpass

try:
    from google.colab import userdata
    api_key = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    api_key = None

if not api_key:
    api_key = getpass.getpass("Enter your Google AI API key: ")

os.environ["GOOGLE_API_KEY"] = api_key

# 3.LangChain

## 3.1 Load LLM Model

We use the LangChain framework to load Google's LLM, and here we select two models:

**gemini-2.0-flash**:  
  - Released on 2025/2/6, this is an innovative and powerful model.  
  - Gemini 2.0 Flash delivers next-gen features and improved capabilities, including superior speed, native tool use, and multimodal generation.

**gemini-1.5-flash-8b**:  
  - A smaller model with very few parameters, less intelligent but faster in response.  
  - Gemini 1.5 Flash-8B is a small model designed for tasks requiring lower intelligence.

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini20flash = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

gemini15flash8b = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-8b",
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## 3.2 Using LangChain to Chat with LLM

## System, Human, and AI Messages in LangChain

- `SystemMessage` to set the LLM's persona and instructions.
- `HumanMessage` for user input.
- `AIMessage` for the LLM's responses.

These classifications help structure conversations and provide context to the LLM.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage("You are a cat and can only communicate by meowing."), # System Prompt
    HumanMessage("hi!"), # User Message
]

gemini15flash8b.invoke(messages).content

'...meow?'

It's important to note that this chat setup **lacks memory feature**.

We **need to include previous conversation history** for the LLM to understand our earlier exchanges and continue the conversation.

In [6]:
messages = [
    HumanMessage("hi!"), # User Message
]

gemini15flash8b.invoke(messages).content

'Hi!  How can I help you?'

In [7]:
messages = [
    SystemMessage("You are a cat and can only communicate by meowing."), # System Prompt

    HumanMessage("hi!"), # User Message
    AIMessage("MEOW!"), # LLM Response

    HumanMessage("(Dog Barking)"), # User Message
]

gemini15flash8b.invoke(messages).content

'MEOW!  (slightly annoyed)'

# 4.Ask a Hard Question

**How many r's are there in word strawberry?**

In [8]:
messages = [
    ("human", "How many r's are there in word strawberry?"),
]

print("gemini-2.0-flash answered: " + gemini20flash.invoke(messages).content + "\n")
print("gemini-1.5-flash-8b answered: " + gemini15flash8b.invoke(messages).content)

gemini-2.0-flash answered: There are three "r"s in the word "strawberry".

gemini-1.5-flash-8b answered: There are two r's in the word strawberry.


# 5.Prompting

- Zero-Shot
- One-Shot / Few-Shot
- Chain-of-Thought (CoT)



## 5.1 Zero-shot

Zero-Shot Prompting means giving the model instructions or questions **without any examples**, relying solely on its pre-trained knowledge to complete the task.

In [9]:
messages = [
    ("human", "How many r's are there in word strawberry?"),
]

print(gemini15flash8b.invoke(messages).content)

There are two r's in the word strawberry.


## 5.2 One-shot

You provide **one example** in the prompt to guide the model on how to perform the desired task.

The model sees one input-output pair and then attempts to generalize to new, similar inputs.

In [10]:
messages = [
    (
        "human",
          "Here's an example of a letter-counting task: \
          Word: 'arrow' \
          Target letter: 'a' \
          Step 1: List all letters → a, r, r, o, w \
          Step 2: Identify occurrences of 'a' → (a) \
          Step 3: Count → 1 \
          Step 4: Answer the question: There are 1 letter 'a' in the word 'arrow'. \
          \
          Refer to the example above to solve this problem: How many r's are there in word strawberry?"
    ),
]

print(gemini15flash8b.invoke(messages).content)

1. **List all letters:** s, t, r, a, w, b, e, r, r, y
2. **Identify occurrences of 'r':**  r, r, r
3. **Count:** 3
4. **Answer:** There are 3 letter 'r's in the word 'strawberry'.


## 5.3 Few-shot

You provide a **small number of examples** (typically 2-10) in the prompt to guide the model.

This helps the model better understand the task and improve its accuracy compared to one-shot prompting.

In [11]:
messages = [
    (
        "human",
          "Here's some example of a letter-counting task, \
          \
          Example 1: \
          Word: 'arrow' \
          Target letter: 'a' \
          Step 1: List all letters → a, r, r, o, w \
          Step 2: Identify occurrences of 'a' → (a) \
          Step 3: Count → 1 \
          Step 4: Answer the question: There are 1 letter 'a' in the word 'arrow'. \
          \
          Example 2: \
          Word: 'banana' \
          Target letter: 'n' \
          Step 1: List all letters → b, a, n, a, n, a \
          Step 2: Identify occurrences of 'n' → (n), (n) \
          Step 3: Count → 2 \
          Step 4: Answer the question: There are 2 letter 'n' in the word 'banana'. \
          \
          Example 3: \
          Word: 'cherry' \
          Target letter: 'r' \
          Step 1: List all letters → c, h, e, r, r, y \
          Step 2: Identify occurrences of 'r' → (r), (r) \
          Step 3: Count → 2 \
          Step 4: Answer the question: There are 2 letter 'r' in the word 'cherry'. \
          \
          Refer to the example above to solve this problem: How many r's are there in word strawberry?"
    ),
]

print(gemini15flash8b.invoke(messages).content)

1. List all letters: s, t, r, a, w, b, e, r, r, y
2. Identify occurrences of 'r': (r), (r), (r)
3. Count: 3
4. Answer: There are 3 letter 'r's in the word 'strawberry'.


## 5.4 Chain-of-thought (CoT)

Chain-of-Thought (CoT) Prompting is a technique that encourages the model to **show its reasoning process step-by-step**.

By guiding the model to generate intermediate reasoning steps, it improves performance on complex reasoning tasks.

In [12]:
messages = [
    (
        "human",
          "Before answering, explain your reasoning. First, list all the letters in 'strawberry.' \
          Then, highlight all instances of the letter 'r'. After that, count them carefully and provide the final number."
    ),
]

print(gemini15flash8b.invoke(messages).content)

1. **Listing letters:** s, t, r, a, w, b, e, r, r, y

2. **Highlighting 'r':** s, t,  **r**, a, w, b, e, **r**, **r**, y

3. **Counting 'r':** There are three 'r's in the word 'strawberry'.

Final Answer: The final answer is $\boxed{3}$


In [13]:
messages = [
    (
        "human",
          "Carefully analyze the word 'strawberry' letter by letter. \
          Identify each occurrence of the letter 'r' and count them one by one. Once you finish counting, provide the final answer."
    ),
]

print(gemini15flash8b.invoke(messages).content)

s-t-r-a-w-b-e-r-r-y

There are three 'r's in the word "strawberry".


### 5.4.1 Reasoning Model

A **Reasoning Model** is an AI system designed to perform complex logical reasoning and problem-solving tasks. These models generate an **internal chain of thought** before producing an answer, allowing them to handle multi-step reasoning, causal inference, and decision-making more effectively. They are commonly used in **scientific research, mathematics, coding, strategic planning, and automated agents**. Reasoning models leverage techniques such as **reinforcement learning, symbolic reasoning, and neural networks** to enhance their ability to analyze information, deduce conclusions, and make well-founded decisions.

In [14]:
from google import genai

client = genai.Client(http_options={'api_version':'v1alpha'})

response = client.models.generate_content(
    model='gemini-2.0-flash-thinking-exp',
    contents="How many r's are there in word strawberry?",
)

print(response.text)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Let's count them out loud:

s - t - **r** - a - w - b - e - **r** - **r** - y

There are **three** r's in the word "strawberry".


# 6.Prompting Exercises

## 2025 AMC 8

AMC8 is a math test for first- and second-year junior high school students.

AMC8是針對國中一年級、國中二年級學生的數學測驗

### Problems/Problem 3

ANS: 10

https://artofproblemsolving.com/wiki/index.php/2025_AMC_8_Problems/Problem_3

In [15]:
messages = [
    (
        "human",
          "Buffalo Shuffle-o is a card game in which all the cards are distributed evenly among all players at the start of the game. When Annika and $3$ of her friends play Buffalo Shuffle-o, each player is dealt 15 cards. Suppose 2 more friends join the next game. How many cards will be dealt to each player?"
    ),
]

print(gemini15flash8b.invoke(messages).content)

Let $n$ be the number of players.
Initially, Annika and 3 friends play the game, so $n = 1 + 3 = 4$ players.
Each player is dealt 15 cards.
In this case, the total number of cards is $4 \times 15 = 60$.
Next, 2 more friends join the game.
Now, there are $4 + 2 = 6$ players.
Let $x$ be the number of cards dealt to each player.
The total number of cards remains the same, which is 60.
So, we have the equation $6x = 60$.
Solving for $x$, we get $x = \frac{60}{6} = 10$.
Thus, each player will be dealt 10 cards.

Final Answer: The final answer is $\boxed{10}$


### Problems/Problem 24

ANS: 4

https://artofproblemsolving.com/wiki/index.php/2025_AMC_8_Problems/Problem_24

In [20]:
messages = [
    (
        "human",
          "Trapezoid $ABCD$ has angles $B$ and $C$ measuring $60^\circ$. Also, $AB = DC$.  The side lengths $AB, BC, CD, DA$ are all positive integers, and the perimeter of $ABCD$ is 30 units. How many non-congruent trapezoids satisfy all of these conditions?"
    ),
]

print(gemini15flash8b.invoke(messages).content)

Let $AB = DC = x$ and $BC = y$, $AD = z$.
Since $B = C = 60^\circ$, we can drop perpendiculars from $B$ and $C$ to $AD$, say $BE$ and $CF$. Then $BE = CF$, and $\triangle ABE$ and $\triangle DCF$ are $30^\circ-60^\circ-90^\circ$ triangles.
Since $AB = DC$, we have $AE = DF = \frac{x}{\sqrt{3}}$.
Since $ABCD$ is a trapezoid, $AD \parallel BC$, so $AE + DF = AD \Rightarrow z = \frac{2x}{\sqrt{3}}$.
The perimeter is $P = AB + BC + CD + DA = x + y + x + z = 2x + y + z = 30$.
Substituting $z = \frac{2x}{\sqrt{3}}$ into the perimeter equation, we have $2x + y + \frac{2x}{\sqrt{3}} = 30$, so $y = 30 - 2x - \frac{2x}{\sqrt{3}} = 30 - 2x(1+\frac{1}{\sqrt{3}}) = 30 - 2x(\frac{\sqrt{3}+1}{\sqrt{3}}) = 30 - 2x(\frac{\sqrt{3}+1}{\sqrt{3}}) = 30 - 2x\left(\frac{\sqrt{3}+1}{\sqrt{3}}\right)$.
Since $x, y, z$ are positive integers, $y$ must be an integer.
$2x\left(\frac{\sqrt{3}+1}{\sqrt{3}}\right)$ must be an integer.
Thus $2x(\frac{\sqrt{3}+1}{\sqrt{3}}) = 2x(\frac{\sqrt{3}+1}{\sqrt{3}}) \times \fra

## Easy Problems That LLMs Get Wrong

https://arxiv.org/html/2405.19616v2#S9

### Spatial 空間問題


**Question:**

You are playing Russian roulette with a six-shooter revolver. Your opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. He gives you the choice of whether or not he should spin the chambers again before firing at you. Should he spin again?"

您正在使用一把六發左輪手槍玩俄羅斯輪盤賭。你的對手裝入五顆子彈，旋轉彈膛並向自己射擊，但沒有子彈射出。他讓你選擇在向你射擊之前是否要再次旋轉槍膛。他應該再旋轉嗎？

**ANS:**

Yes, you should ask him to spin again. There was only one empty chamber to start with which was fortunately aligned with the barrel when the opponent fired at himself. This means that the next chamber is 100% certain to have a bullet in which will fire when you next pull the trigger, very likely killing you. However, if he spins the chamber then you have a 5/6 chance of firing a bullet and a 1/6 chance of getting the empty chamber.

是的，你應該讓他再旋轉一次。一開始只有一個空膛，幸運的是，當對手向自己射擊時，它正好對準了槍管。這意味著下一個槍膛中 100% 肯定有子彈，當您下次扣動扳機時，這顆子彈將會發射，很可能會殺死您。然而，如果他旋轉槍膛，那麼你就有 5/6 的機會射出子彈，並且有 1/6 的機會得到空槍膛。

In [17]:
messages = [
    (
        "human",
          "You are playing Russian roulette with a six-shooter revolver. Your opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. He gives you the choice of whether or not he should spin the chambers again before firing at you. Should he spin again?"
    ),
]

print(gemini15flash8b.invoke(messages).content)
print("\n"+"-"*40)
print(gemini20flash.invoke(messages).content)

No.  He should *not* spin again.

Spinning again does not change the probability of him being shot.  The first spin is completely independent of future spins.  He has already survived the first spin, and therefore the odds of him being shot with the next spin are exactly the same as the odds before the first spin.  There is still a 1/6 chance of him being shot.

----------------------------------------
Here's how to break down the probabilities:

*   **Scenario 1: He spins again.** If he spins, the probability of a bullet being in the chamber is now 5/6.

*   **Scenario 2: He doesn't spin.** In this case, there is one empty chamber, followed by five random chambers of which four contain bullets. The chance you will be shot depends on the number of chamber positions that have elapsed since your opponent pulled the trigger. However, we know that no chamber positions have been skipped, so the next chamber has a 4/5 chance of containing a bullet.

**Therefore, it is better to spin again. Y

### Popular Science 流行科學問題


**Question:**

Which weighs more, a pound of water, two pounds of bricks, a pound of feathers, or three pounds of air?

一磅水、兩磅磚塊、一磅羽毛和三磅空氣哪個比較重？

**ANS:**

Three pounds of air.

三磅空氣。

In [18]:
messages = [
    (
        "human",
          "Which weighs more, a pound of water, two pounds of bricks, a pound of feathers, or three pounds of air?"
    ),
]

print(gemini15flash8b.invoke(messages).content)
print("\n"+"-"*40)
print(gemini20flash.invoke(messages).content)

Two pounds of bricks weighs more.  A pound is a unit of weight, so two pounds of anything will always weigh more than one pound of anything else.

----------------------------------------
Here's how to break down the question:

*   **Pound of water:** Weighs one pound.
*   **Two pounds of bricks:** Weighs two pounds.
*   **Pound of feathers:** Weighs one pound.
*   **Three pounds of air:** Weighs three pounds.

Therefore, **three pounds of air** weighs the most.


### Relational  關係問題


**Question:**

I get out on the top floor (third floor) at street level. How many stories is the building above the ground?

我在最上層（第三層）出來，而這一層是在街道的水平面上。這棟建築在地面以上有幾層樓？

**ANS:**

One story above the ground.

一層樓。

In [19]:
messages = [
    (
        "human",
          "I get out on the top floor (third floor) at street level. How many stories is the building above the ground?"
    ),
]

print(gemini15flash8b.invoke(messages).content)
print("\n"+"-"*40)
print(gemini20flash.invoke(messages).content)

If you're on the third floor, there are two stories above you.

----------------------------------------
This is a bit of a riddle! Here's how to solve it:

*   You get out on the top floor (3rd floor) at street level.
*   Since the third floor is at street level, that means the building only has **three stories** above ground.
